In [ ]:
import numpy as np
import sys
import os
import glob
import matplotlib.pyplot as plt
import h5py
import cv2
import gc
import bat_functions as kbf
from multiprocessing import Pool

In [ ]:
track_folder = ".../kasanka-bats/processed/deep-learning"
day = "16Nov"
camera_folders = sorted(glob.glob(os.path.join(track_folder, day, '*')))

In [ ]:
camera_folders

In [ ]:
observation_folders = glob.glob(
    '.../kasanka-bats/processed/deep-learning/16Nov/*/raw_tracks.npy')
print(len(observation_folders))

In [ ]:
# for camera_folder in camera_folders:
#     print(os.path.basename(camera_folder))
#     print(os.path.exists(os.path.join(camera_folder, 'centers.npy')))

In [ ]:
n_camera_folders = []
for folder in camera_folders:
    tracks_file = os.path.join(folder, 'raw_tracks.npy')
    if os.path.exists(tracks_file):
        print(tracks_file)
    else:
        n_camera_folders.append(folder)
# n_camera_folders.extend(camera_folders[2:5])
# n_camera_folders.append(camera_folders[6])
# n_camera_folders.extend(camera_folders[8:])
print('to track')
print(*n_camera_folders, sep='\n')

In [ ]:
# camera_folders = n_camera_folders
n_camera_folders = n_camera_folders[:]
n_camera_folders

In [ ]:
# def get_obs_ind_from_file(elem):
#     return int(elem.split('_')[-1].split('.')[0])

# num_contour_files = 15

# for camera_folder in camera_folders[:]:
#     contours = np.load(os.path.join(camera_folder, 'contours.npy'), allow_pickle=True)
# #     centers = np.load(os.path.join(camera_folder, 'centers.npy'), allow_pickle=True)
# #     rects = np.load(os.path.join(camera_folder, 'rects.npy'), allow_pickle=True)
# #     sizes = np.load(os.path.join(camera_folder, 'size.npy'), allow_pickle=True)
# #     frame_folder = os.path.join(camera_folder, 'example-frames')
# #     frame_files = sorted(glob.glob(os.path.join(frame_folder, '*.jpg')), key=get_obs_ind_from_file)

#     new_contours = []
#     file_num = 0
#     for frame_ind, cs in enumerate(contours):
#         if frame_ind % int(len(contours)/num_contour_files) == 0:
#             # start new file
#             file_name = f'contours-compressed-{file_num:02d}.npy'
#             file = os.path.join(camera_folder, file_name)
#             np.save(file, np.array(new_contours, dtype=object))
#             new_contours = []
#             file_num += 1
#         if frame_ind % 10000 == 0:
#             print(frame_ind)
#         new_contours.append([])
#         for c in cs:
#             cc	= np.squeeze(cv2.approxPolyDP(c, 0.1, closed=True))
#             new_contours[-1].append(cc)
#     file_name = f'contours-compressed-{file_num:02d}.npy'
#     file = os.path.join(camera_folder, file_name)
#     np.save(file, np.array(new_contours, dtype=object))

In [ ]:
# x = sorted(
#             glob.glob(os.path.join(camera_folders[5], 'contours-compressed-*.npy'))
#         )
# centers = np.load(os.path.join(camera_folders[5], 'centers.npy'), allow_pickle=True)

In [ ]:
# ind = 4
# print(len(centers[ind]), len(c[ind]))

In [ ]:
# c = np.load(x[1], allow_pickle=True)

In [ ]:
# len(c)
# len(c[100])

In [ ]:
def track(camera_dict):
#     try:
    camera_folder = camera_dict['camera_folder']
    first_frame = camera_dict['first_frame']
    max_frame = camera_dict['max_frame']
    print('{} begun. '.format(os.path.basename(camera_folder)))
    contours_files = sorted(
        glob.glob(os.path.join(camera_folder, 'contours-compressed-*.npy'))
    )
    if contours_files:
        contours_files = contours_files[1:]
        centers = np.load(os.path.join(camera_folder, 'centers.npy'), allow_pickle=True)
        sizes = np.load(os.path.join(camera_folder, 'size.npy'), allow_pickle=True)
        tracks_file = os.path.join(camera_folder, f'first_frame_{first_frame}_max_val_{max_frame}_raw_tracks.npy')
        raw_tracks = kbf.find_tracks(first_frame, centers, contours_files=contours_files, 
                                     sizes_list=sizes, tracks_file=tracks_file,
                                    max_frame=max_frame)
#     except:
#         print(camera_folder, 'failed.')

In [ ]:
n_camera_folders

In [ ]:
camera_dicts = []
for camera_folder in n_camera_folders:
    centers_file = os.path.join(camera_folder, 'centers.npy')
    centers = np.load(centers_file, allow_pickle=True)
    max_vals = np.linspace(0, len(centers), 10, dtype=int)[1:].tolist()
    max_vals[-1] = None
    min_vals = np.linspace(0, len(centers), 10, dtype=int)[:-1]
    # 15 second overlap
    min_vals[1:] = min_vals[1:] - 450
    for min_val, max_val in zip(min_vals, max_vals):
        min_val = np.max([min_val, 0])
        camera_dict = {'camera_folder': camera_folder,
                       'first_frame': min_val,
                       'max_frame': max_val}
        if max_val is None:
            tracks_basename = f'first_frame_{min_val:06d}_max_val_{max_val}_raw_tracks.npy'
        else:
            tracks_basename = f'first_frame_{min_val:06d}_max_val_{max_val:06d}_raw_tracks.npy'
        tracks_file = os.path.join(camera_folder, tracks_basename)
        if not os.path.exists(tracks_file):
            print(tracks_file)
            camera_dicts.append(camera_dict)
        

In [ ]:
print(camera_folder)
print(camera_dicts)

In [ ]:
first_frame_ind = 100

contours_files = sorted(
        glob.glob(os.path.join(camera_folder, 'contours-compressed-*.npy'))
    )
if contours_files:
    contours_files = contours_files[1:]
    centers = np.load(os.path.join(camera_folder, 'centers.npy'), allow_pickle=True)

contours_file_ind = 0
contours_list = np.load(contours_files[contours_file_ind], allow_pickle=True)
previous_contours_seen = 0
while first_frame_ind >= previous_contours_seen + len(contours_list):
    contours_file_ind += 1
    previous_contours_seen += len(contours_list)
    contours_list = np.load(contours_files[contours_file_ind], allow_pickle=True)
print(f'using {contours_files[contours_file_ind]}')    

contours_ind = first_frame_ind - previous_contours_seen - 1

In [ ]:
first_frame_ind = 100

# for first_frame_ind in range(first_frame_ind, first_frame_ind + 1000):
contours_ind = first_frame_ind - previous_contours_seen - 1
if len(contours_list[contours_ind+1]) != len(centers[first_frame_ind]):
    print(first_frame_ind)

In [ ]:
print(os.path.join(gopro_folder, camera_name, '*', 'contours-compressed-*.npy'))

In [ ]:
for camera_folder in n_camera_folders[:1]:
    print(os.path.basename(camera_folder))
    centers = np.load(os.path.join(camera_folder, 'centers.npy'), allow_pickle=True)
    contours_files = sorted(
            glob.glob(os.path.join(camera_folder, 'contours-compressed-*.npy'))
        )
    contours = [np.load(f, allow_pickle=True) for f in contours_files]
    camera_name = os.path.basename(camera_folder)
    
    contour_lengths = [len(c) for c in contours]
    print(contour_lengths)
    print(sum(contour_lengths), len(centers), len(contours_files))

In [ ]:
centers = np.zeros((60930, 2))
contours_list = [[0,1,2,3] for i in range(centers.shape[0])]
print(len(centers), len(contours_list))

In [ ]:
tracks = np.load(tracks_file, allow_pickle=True)

In [ ]:
len(tracks)

In [ ]:
camera_dicts

In [ ]:
with Pool(processes=5) as pool:
    pool.map(track, camera_dicts)

In [ ]:
glob.glob(file)

In [ ]:
def get_obs_ind_from_file(elem):
    return int(elem.split('_')[-1].split('.')[0])

# num_contour_files = 15

for camera_folder in camera_folders[5:6]:

    frame_folder = os.path.join(camera_folder, 'example-frames')
    frame_files = sorted(glob.glob(os.path.join(frame_folder, '*.jpg')), key=get_obs_ind_from_file)
    centers = np.load(os.path.join(camera_folder, 'centers.npy'), allow_pickle=True)

In [ ]:
len(frame_files), len(centers)

In [ ]:
plt.figure(figsize=(20,10))
example_frame_ind = 80
shift = 48
obs_ind = int(frame_files[example_frame_ind].split('_')[-1].split('.')[0])
im = plt.imread(frame_files[example_frame_ind])
plt.imshow(im, vmin=0, vmax=255)
plt.colorbar()
plt.scatter(shift + centers[obs_ind][:,0], shift + centers[obs_ind][:,1], c='r', s=.1)
x_border = [shift, shift, im.shape[1] - shift, im.shape[1] - shift, shift]
y_border = [shift, im.shape[0] - shift, im.shape[0] - shift, shift, shift]
plt.plot(x_border, y_border)
plt.figure(figsize=(20,10))
plt.imshow(im)

plt.colorbar()
plt.scatter(shift + centers[obs_ind][:,0], shift + centers[obs_ind][:,1], c='r', s=1)
x_border = [shift, shift, im.shape[1] - shift, im.shape[1] - shift, shift]
y_border = [shift, im.shape[0] - shift, im.shape[0] - shift, shift, shift]
plt.plot(x_border, y_border)

In [ ]:
plt.figure(figsize=(20,10))
example_frame_ind = 0
shift = 48
obs_ind = int(frame_files[example_frame_ind].split('_')[-1].split('.')[0])
im = plt.imread(frame_files[example_frame_ind])
plt.imshow(im, vmin=0, vmax=255)
plt.colorbar()
plt.scatter(shift + centers[obs_ind][:,0], shift + centers[obs_ind][:,1], c='r', s=.1)
x_border = [shift, shift, im.shape[1] - shift, im.shape[1] - shift, shift]
y_border = [shift, im.shape[0] - shift, im.shape[0] - shift, shift, shift]
plt.plot(x_border, y_border)
plt.figure(figsize=(20,10))
plt.imshow(im)

plt.colorbar()
plt.scatter(shift + centers[obs_ind][:,0], shift + centers[obs_ind][:,1], c='r', s=1)
x_border = [shift, shift, im.shape[1] - shift, im.shape[1] - shift, shift]
y_border = [shift, im.shape[0] - shift, im.shape[0] - shift, shift, shift]
plt.plot(x_border, y_border)


In [ ]:
# contour_folder = os.path.join(camera_folder, 'contours')
# os.makedirs(contour_folder, exist_ok=True)

In [ ]:
file

In [ ]:
# import time
# t0 = time.time()
# for obs_ind, contour_list in enumerate(contours[0:2000]):
#     file = os.path.join(contour_folder, f'contours_obs-ind_{obs_ind:06}.npy')
#     np.save(file, np.array(contour_list, dtype=object))
# total = time.time() - t0

# print(total, total/obs_ind)

In [ ]:
# contour_files = sorted(glob.glob(os.path.join(contour_folder, '*.npy')))